<a href="https://colab.research.google.com/github/panzershracker/Web-scraping/blob/master/func_tests3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import requests, re
from datetime import datetime, date, time, timedelta
from bs4 import BeautifulSoup as soup
from pprint import pprint



In [0]:
mask = '[а-я0-9—]+'
vacancy = 'программист'
language = 'python'
city = 'moskva.'
salary = '?st=1'
page = '&p=1'

In [0]:
def get_vac_names():

    div = str(soup(resp.text, 'lxml').find_all('div', {'class': ['clearfix vacancy premium ',
                                                                 'clearfix vacancy']}))
    h2 = str(soup(div, 'lxml').find_all('h2'))
    
    a = list(soup(h2, 'lxml').find_all('a'))
    
    vac_names = []

    for i in range(0, len(a)):
        vac_names.append(a[i].getText())

    return vac_names




In [0]:
# print(len(get_vac_names()))

# pprint(get_vac_names())

In [0]:
def time_ago(resp_text):
  
  span = resp_text.find_all('span', {'class': 'date'})
  
  time_ago = []
  
  for i in range(0, len(span)):
    time = span[i].find(text=True, recursive=False).replace('  ', '')[1:]
    time_ago.append(time)
    
  return time_ago
  

In [281]:
pprint(time_ago(resp_text)[:5])

['5 дней назад с',
 '1 неделю назад с',
 '10 часов назад с',
 '9 часов назад с',
 '']


In [0]:
# a = time_ago(resp_text)

# for i in a:
    
#     num, rest = i.split(' ', 1)
    
#     print(num)
#     print(rest[0])

In [0]:
def post_datetime(time_ago, resp_text):
  
  time_ago = time_ago(resp_text)  
 
  date_time = []
  
  for i in time_ago:
    
    now = datetime.now()
      
    if not i:
      date_time.append(0)
      
    else: 

      num, rest = i.split(' ', 1)

      if rest.startswith('ч'):
        date_time.append(now - timedelta(hours= int(num)))

      elif rest.startswith('д'):
        date_time.append(now - timedelta(days= int(num)))

      elif rest.startswith('н'):
        date_time.append(now - timedelta(weeks= int(num)))

      elif rest.startswith('м'):
        date_time.append(now - timedelta(weeks= int(num)*4))

      else:
        date_time.append(0)

    
  return date_time
  

In [0]:
# for i in post_datetime(time_ago, resp_text)[:5]:
#   print(i)

In [0]:
def get_salaries():
   
    h2 = str(soup(resp.text, 'lxml').find_all('h2'))
    
    b = list(soup(h2, 'lxml').find_all('b'))
    
    salaries = []
    
    for i in b:
      sal = re.findall(mask, str(i))
      salaries.append(''.join(sal))
      
    return salaries



In [0]:
def get_links():
  
  div = str(soup(resp.text, 'lxml').find_all('div', {'class': ['clearfix vacancy premium ',
                                                                 'clearfix vacancy']}))
  h2 = str(soup(div, 'lxml').find_all('h2'))
  
  a = list(soup(h2, 'lxml').find_all('a'))
  
  vac_links = []

  for i in range(0, len(a)):
    vac_links.append('http://gorodrabot.ru' + a[i].get('href'))
        

  return vac_links


In [0]:
def hirer():
  
  div = str(soup(resp.text, 'lxml').find_all('div', {'class': ['clearfix vacancy premium ',
                                                                 'clearfix vacancy']}))
  h2 = str(soup(div, 'lxml').find_all('h2'))
  
  a = list(soup(h2, 'lxml').find_all('a'))
  
  hirers = []

  for i in range(0, len(a)):
    hirers.append(a[i].get('data-action'))
        

  return hirers

In [0]:
# pprint(hirer())

In [0]:
def get_comp():
  
  div = str(soup(resp.text, 'lxml').find_all('div', {'class': ['clearfix vacancy premium ',
                                                               'clearfix vacancy']}))
  div = str(soup(div, 'lxml').find_all('div', {'class': 'address'}))
  
  span = list(soup(div, 'lxml').find_all('span'))
  
  comp = []
  
  for i in range(0, len(span)):
    comp.append(span[i].getText())

  return comp

# ! TODO

### Доделать вывод названия компаний.

In [0]:
# pprint(get_comp())

In [0]:
# print(get_links())

In [0]:
def salary_separator():
  
  ot = []
  do = []
  
  salaries = get_salaries()
  
  for i in salaries:
    
    if i.startswith('от'):
      
      ot.append(int(i[2:]))
      do.append(0)
    
    elif i.startswith('до'):
      
      if i.startswith('дог'):
        
        ot.append(-1)
        do.append(-1)
      
      else:
        
        do.append(int(i[2:]))
        ot.append(0)
      
    else:
      
      a = i.split('—')
      
      ot.append(int(a[0]))
      do.append(int(a[1]))
      
  return ot, do

In [257]:
with requests.Session() as session:

    session.headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'

    url = f'https://{city}gorodrabot.ru/{vacancy}_{language}{salary}{page}'
    resp = session.get(url)
    
    resp_text = soup(resp.text, 'lxml')

    print(f'gorodrabot.ru status code is: {resp.status_code}')


gorodrabot.ru status code is: 200


In [0]:
    df = pd.DataFrame({'vacancy': get_vac_names(),
                       'salary min': salary_separator()[0],
                       'salary max': salary_separator()[1],
                       'link': get_links(),
                       'from': hirer(),
                       'datetime': post_datetime(time_ago, resp_text)})

In [307]:
df.head(20)

,vacancy,salary min,salary max,link,from,datetime
0,Разработчик Backend (Middle),180000,250000,http://gorodrabot.ru/vacancy/198182127/razrabo...,rabota.ru,2019-08-14 15:59:58.040258
1,Программист php,20000,40000,http://gorodrabot.ru/vacancy/198682024/program...,rabota.ru,2019-08-12 15:59:58.040291
2,Backend-разработчик (python 3),100000,200000,http://gorodrabot.ru/vacancy/193786676/backend...,talents.yandex.ru,2019-08-19 05:59:58.040297
3,Senior Fullstack разработчик,150000,0,http://gorodrabot.ru/vacancy/199424618/senior_...,talents.yandex.ru,2019-08-19 06:59:58.040301
4,Windows-разработчик со знанием JS,80000,0,http://gorodrabot.ru/vacancy/198424878/windows...,None,0
5,Intelligent R&D Tools Technical Expert,300000,0,http://gorodrabot.ru/vacancy/196539228/intelli...,kellyservices.ru,2019-07-29 15:59:58.040304
6,Разработчик Python,100000,180000,http://gorodrabot.ru/vacancy/199205772/razrabo...,hh.ru,2019-08-18 17:59:58.040308
7,Программист Python,90000,0,http://gorodrabot.ru/vacancy/197245059/program...,hh.ru,2019-08-16 15:59:58.040310
8,Junior Python developer / Начинающий разработч...,60000,70000,http://gorodrabot.ru/vacancy/200153543/junior_...,hh.ru,2019-08-16 15:59:58.040313
9,Программист Python (Middle),100000,0,http://gorodrabot.ru/vacancy/199967520/program...,hh.ru,2019-08-18 17:59:58.040315


In [0]:
# print(len(get_vac_names()))
# print(len(get_salaries()))
# print(len(get_links()))
# print(len(salary_separator()[0]))
# print(len(salary_separator()[1]))